# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"

params = {
    'appid':api_keys.api_key
}

city_name= []
lat =[]
lng = []
cloudiness = []
date = []
mtemp = []
wind = []
humid = []
country = []
i = 0

for c in cities:
    i +=1
    params['q']=c
    response = requests.get(base_url, params = params).json()

    try:
        city_name.append(response['name'])
        country.append(response['sys']['country'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        cloudiness.append(response['clouds']['all'])
        date.append(response['dt'])
        mtemp.append(response['main']['temp_max'])
        humid.append(response['main']['humidity'])
        wind.append(response['wind']['speed'])
        print('City',i, response['name'])
    except:
        print('City',i,'not found. Skipping...')

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:

weather_df = pd.DataFrame(data={'City Name':city_name, 'Country':country, 'Lat':lat, 'Lng':lng, 'Date':date,
                                'Humidity':humid, 'Cloudiness':cloudiness, 'Max Temp':mtemp, 'Wind Speed':wind})
weather_df['Max Temp'] = (weather_df['Max Temp']-273)*9/5 + 32
weather_df.to_csv(output_data_file)
weather_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
today = datetime.datetime.today().strftime('%m/%d/%Y')
plt.scatter(weather_df['Lat'],weather_df['Max Temp'],alpha =0.7,edgecolors='black')
plt.grid()
plt.ylabel('Temperature (F)')
plt.xlabel('Latitude')
plt.title(f'City Latitude vs. Temperature ({today})')
plt.savefig('LatTempPlot.png')
plt.show()


#### Latitude vs. Humidity Plot

In [ ]:
today = datetime.datetime.today().strftime('%m/%d/%Y')
plt.scatter(weather_df['Lat'],weather_df['Humidity'],alpha =0.7,edgecolors='black')
plt.grid()
plt.title(f'City Latitude vs. Humidity ({today})')
plt.ylabel('Humidity (%)')
plt.xlabel('Latitude')
plt.savefig('LatHumidityPlot.png')
plt.show()


#### Latitude vs. Cloudiness Plot

In [ ]:
today = datetime.datetime.today().strftime('%m/%d/%Y')
plt.scatter(weather_df['Lat'],weather_df['Cloudiness'],alpha =0.7,edgecolors='black')
plt.grid()
plt.title(f'City Latitude vs. Cloudiness ({today})')
plt.ylabel('Cloudiness (%)')
plt.xlabel('Latitude')
plt.savefig('LatCloudinessPlot.png')
plt.show()


#### Latitude vs. Wind Speed Plot

In [ ]:
today = datetime.datetime.today().strftime('%m/%d/%Y')
plt.scatter(weather_df['Lat'],weather_df['Wind Speed'],alpha =0.7,edgecolors='black')
plt.grid()
plt.title(f'City Latitude vs. Wind Speed ({today})')
plt.ylabel('WindSpeed (mph)')
plt.xlabel('Latitude')
plt.savefig('LatWindPlot.png')
plt.show()
